# Generative Adversarial Network 

 > - Generator와 Discriminator, 두 뉴럴넷이 서로 경쟁하면서 결과적으로 Discriminator의 결과가 0.5(구별 하지 못한다.)에 수렴하면서, Generator가 실제 데이터와 흡사한 데이터를 만들 수 가 있다.  
 
 - 여러 적용 사례 ; CycleGAN, StarGAN 등 ... 
 
 
###### Generator 
- 난수를 입력 받아, real image와 동일한 이미지를 출력하는 Neural Network 
- Discriminator 의 구분능렬이 좋을 수록 Generator의 loss가 증가 
- 실제로 이 Generator를 학습 시켜서 이미지를 생성하는 모델을 만드는 것이 목표 

###### Discriminator 
- fake image와 real image를 분류하는 Neural Network 
- fake image는 0, real_image는 1

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 

/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

def generator(z, reuse=False):
    name = 'generator/layer{}'
    layer1 = tf.layers.dense(z, 10, name=name.format(1), reuse=reuse)
    layer2 = tf.layers.dense(layer1, 10, name=name.format(2), reuse=reuse)
    layer3 = tf.layers.dense(layer2, 10, name=name.format(3), reuse=reuse)
    output = tf.layers.dense(layer3, 10, name=name.format('out'), reuse=reuse)
    return output

def discriminator(feature, reuse=False):
    name = 'discriminator/layer{}'
    layer1 = tf.layers.dense(feature, 10,  name=name.format(1), reuse=reuse)
    layer2 = tf.layers.dense(layer1, 10, name=name.format(2), reuse=reuse)
    layer3 = tf.layers.dense(layer2, 10, name=name.format(3), reuse=reuse)
    layer4 = tf.layers.dense(layer3, 10, name=name.format(4), reuse=reuse)
    output = tf.layers.dense(layer4, 1, name=name.format('output'), reuse=reuse)
    return output

def gan_loss(logits_real, logits_fake):
    loss_real = tf.losses.sigmoid_cross_entropy(tf.ones_like(logits_real), logits_real)
    loss_fake = tf.losses.sigmoid_cross_entropy(tf.zeros_like(logits_fake), logits_fake)
    discriminator_loss = loss_real + loss_fake
    generator_loss = tf.losses.sigmoid_cross_entropy(tf.ones_like(logits_fake), logits_fake)
    return generator_loss, discriminator_loss

def train(loss, variables):
    optimizer = tf.train.AdadeltaOptimizer(1e-2)
    grads = optimizer.compute_gradients(loss, var_list=variables)
    return optimizer.apply_gradients(grads)

data = [[i for i in range(10)] for _ in range(1000)]
normal = np.random.randn(1000, 10)

real_data = tf.placeholder(tf.float32, [None, 10])
z = tf.placeholder(tf.float32, [None, 10])

fake_data = generator(z)
disc_real_logits = discriminator(real_data, False)
disc_fake_logits = discriminator(fake_data, True)

disc_real_prob = tf.reduce_mean(tf.nn.sigmoid(disc_real_logits))
disc_fake_prob = tf.reduce_mean(tf.nn.sigmoid(disc_fake_logits))

loss_gen, loss_disc = gan_loss(disc_real_logits, disc_fake_logits)

variables = tf.trainable_variables()

gen_variables = [v for v in variables if v.name.startswith('generator')]
disc_variables = [v for v in variables if v.name.startswith('discriminator')]

print('variables in generator ------------')
for v in gen_variables:
    print(v)

print('variables in discriminator --------')
for v in disc_variables:
    print(v)
    
disc_train_op = train(loss_disc, disc_variables)
gen_train_op = train(loss_gen, gen_variables)

variables in generator ------------
<tf.Variable 'generator/layer1/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'generator/layer1/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'generator/layer2/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'generator/layer2/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'generator/layer3/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'generator/layer3/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'generator/layerout/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'generator/layerout/bias:0' shape=(10,) dtype=float32_ref>
variables in discriminator --------
<tf.Variable 'discriminator/layer1/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'discriminator/layer1/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'discriminator/layer2/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'discriminator/layer2/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'discriminator/layer3/ker

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        _, _real_prob, _fake_prob = sess.run([disc_train_op, disc_real_prob, disc_fake_prob],
                                            feed_dict={real_data: data, z: normal})
        
        _, _loss = sess.run([gen_train_op, loss_gen], feed_dict={z:normal})
        
        if i % 100 == 0:
            print(f'step:{i} real_prob:{_real_prob}, fake prob:{_fake_prob} loss{_loss}')
            
    _gen = sess.run(fake_data, feed_dict={z:normal})
    print('generating data')
    for g in _gen[:10]:
        print(g)

step:0 real_prob:0.012575588189065456, fake prob:0.49468445777893066 loss0.7289282083511353
step:100 real_prob:0.015554897487163544, fake prob:0.4971252977848053 loss0.7206860780715942
step:200 real_prob:0.01958148181438446, fake prob:0.49971333146095276 loss0.7124038338661194
step:300 real_prob:0.025030065327882767, fake prob:0.5024200081825256 loss0.7042175531387329
step:400 real_prob:0.03247089311480522, fake prob:0.5052425265312195 loss0.696163535118103
step:500 real_prob:0.042734481394290924, fake prob:0.5081848502159119 loss0.688257098197937
step:600 real_prob:0.05702781304717064, fake prob:0.5112552046775818 loss0.6805002689361572
step:700 real_prob:0.07708130776882172, fake prob:0.5144596099853516 loss0.6728922128677368
step:800 real_prob:0.10528557747602463, fake prob:0.5177991986274719 loss0.665417492389679
step:900 real_prob:0.14468558132648468, fake prob:0.5212713479995728 loss0.6580614447593689
generating data
[ 0.07950938  0.24117716  0.35763004  0.12281908 -0.19235767  0